<h2 id="image-gradient-vector">Image Gradient Vector</h2>

<p>First of all, I would like to make sure we can distinguish the following terms. They are very similar, closely related, but not exactly the same.</p>

<table class="info">
  <tbody>
    <tr>
      <td> </td>
      <td><strong>Derivative</strong></td>
      <td><strong>Directional Derivative</strong></td>
      <td><strong>Gradient</strong></td>
    </tr>
    <tr>
      <td>Value type</td>
      <td>Scalar</td>
      <td>Scalar</td>
      <td>Vector</td>
    </tr>
    <tr>
      <td>Definition</td>
      <td>The rate of change of a function <script type="math/tex">f(x,y,z,...)</script> at a point <script type="math/tex">(x_0,y_0,z_0,...)</script>, which is the slope of the tangent line at the point.</td>
      <td>The instantaneous rate of change of <script type="math/tex">f(x,y,z, ...)</script> in the direction of an unit vector <script type="math/tex">\vec{u}</script>.</td>
      <td>It points in the direction of the greatest rate of increase of the function, containing all the partial derivative information of a multivariable function.</td>
    </tr>
  </tbody>
</table>